In [41]:
# import sys
# !{sys.executable} -m pip install tensorflow

In [15]:
#modules for Keras gym and tensorflow
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [16]:
#data fetch 
data=pd.read_csv('stock_data_combined.csv')

In [17]:
#creating positive, nagative and neutral data
positive=[]
negative=[]
neutral=[]
for value in data.Sentiment:
    if value == 0:
        positive.append(0)
        neutral.append(0)
        negative.append(1)
    elif value == 0.5:
        positive.append(0)
        neutral.append(1)
        negative.append(0)
    elif value == 1:
        positive.append(1)
        neutral.append(0)
        negative.append(0)    
        
data['Positive']=positive
data['Negative']=negative
data['Neutral']=neutral
data.head(20)

,Text,Sentiment,Positive,Negative,Neutral
0,- any day now. When it gets below 61.60.,0.0,0,1,0
1,"Elephant hunting ISG, PCN, AMZN, A point and...",0.0,0,1,0
2,FDX double toppoing BEAISH MONTHY GATEY pat...,0.0,0,1,0
3,ISG An update to our Feb 20th video review.....,0.0,0,1,0
4,----------------------------------------------...,0.5,0,0,1
5,On Jan 24th we posted this video on aapl.......,0.0,0,1,0
6,"PAT ONE ES, SPY update",0.0,0,1,0
7,"SPY,IYT, FDX, PS A closer examination of tra...",0.0,0,1,0
8,The rodeo clown sent BK screaming into the S...,0.0,0,1,0
9,We explain in the Jan 24th aapl video how no...,0.0,0,1,0


### Clean Data

In [18]:
# import sys
# !{sys.executable} -m pip install nltk

In [19]:
#making sentance into tokens and cleaning the tokens by removing symbols
from nltk import word_tokenize, WordNetLemmatizer
# import nltk
# nltk.download('punkt')
unclean_tokens = [word_tokenize(sen) for sen in data.Text]
tokens=[]
remove_sym=[';',':','!','@','#','$','%',',','.','^','&','*','(',')','/','?','`','...','..','--','---','~',"+","=",'-',"}","{"]
for i in unclean_tokens:
    
    for j in i:
            if j in remove_sym:
                i.remove(j)
    tokens.append(i)            
tokens

[['any', 'day', 'now', 'When', 'it', 'gets', 'below', '61.60'],
 ['Elephant',
  'hunting',
  'ISG',
  'PCN',
  'AMZN',
  'A',
  'point',
  'and',
  'figure',
  'review',
  'high',
  'risk',
  'define',
  'loss',
  'w',
  'opt'],
 ['FDX',
  'double',
  'toppoing',
  'BEAISH',
  'MONTHY',
  'GATEY',
  'pattern',
  '....',
  'time',
  'to',
  'exit'],
 ['ISG',
  'An',
  'update',
  'to',
  'our',
  'Feb',
  '20th',
  'video',
  'review',
  'if',
  'it',
  'closes',
  'below',
  '495',
  'much',
  'lower',
  'to',
  'come',
  'soon'],
 ['--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  '--',
  'Munich',
  '14',
  'January',
  '2008',
  'BAVARIA',
  'Industriekapital',
  'AG',
  'closed',
  'the',
  'acquisition',
  'of',
  'Elcoteq',
  'Communications',
  'Technology',
  'GmbH',
  'in',
  'Offenburg',
  'Germany',
  'with',
  'the',
  'approval',
  'of',
  'the'],
 ['On',
  'Jan',


In [20]:
#converting tokens from Uppercase to Lowercase
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 

In [23]:
#downloads most commonly used words form Natural Lang toolkit and remove them from tokens
from nltk.corpus import stopwords
import pickle
# nltk.download('stopwords')
stoplist = stopwords.words('english')
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stoplist]
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 
print(stoplist)
with open("stop_list", "wb") as fp:   #Pickling
    pickle.dump(stoplist, fp)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [24]:
result = [' '.join(sen) for sen in filtered_words]
# result

In [25]:
data['Text_Final'] = result
data['tokens'] = filtered_words
data

,Text,Sentiment,Positive,Negative,Neutral,Text_Final,tokens
0,- any day now. When it gets below 61.60.,0.0,0,1,0,day gets 61.60,"[day, gets, 61.60]"
1,"Elephant hunting ISG, PCN, AMZN, A point and...",0.0,0,1,0,elephant hunting isg pcn amzn point figure rev...,"[elephant, hunting, isg, pcn, amzn, point, fig..."
2,FDX double toppoing BEAISH MONTHY GATEY pat...,0.0,0,1,0,fdx double toppoing beaish monthy gatey patter...,"[fdx, double, toppoing, beaish, monthy, gatey,..."
3,ISG An update to our Feb 20th video review.....,0.0,0,1,0,isg update feb 20th video review closes 495 mu...,"[isg, update, feb, 20th, video, review, closes..."
4,----------------------------------------------...,0.5,0,0,1,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"[--, --, --, --, --, --, --, --, --, --, --, -..."
...,...,...,...,...,...,...,...
10497,"ZOOM long .76, looking for continuation/momo",1.0,1,0,0,zoom long .76 looking continuation/momo,"[zoom, long, .76, looking, continuation/momo]"
10498,"ZOT good entry here, next fib level is 10.39, ...",1.0,1,0,0,zot good entry next fib level 10.39 current su...,"[zot, good, entry, next, fib, level, 10.39, cu..."
10499,ZOT Sold at 10.99 the shares I bought Mar 1 fo...,1.0,1,0,0,zot sold 10.99 shares bought mar 1 8.8,"[zot, sold, 10.99, shares, bought, mar, 1, 8.8]"
10500,ZQK first sign of decreasing volume and 3 (15)...,0.0,0,1,0,zqk first sign decreasing volume 3 15 minute r...,"[zqk, first, sign, decreasing, volume, 3, 15, ..."


### Split Data 

In [26]:
data_train, data_test = train_test_split(data, test_size=0.15, random_state=41)

In [14]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
TRAINING_VOCAB
import pickle
with open("TRAINING_VOCAB", "wb") as fp:   #Pickling
    pickle.dump(TRAINING_VOCAB, fp)

KeyError: 'tokens'

In [53]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))

### World2Vec Model algos

In [56]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [57]:
word2vec

In [58]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [59]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

### tokenize and pad sequence

In [140]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))
TEXT_DATA=data_train["Text_Final"].tolist()

Found 14880 unique tokens.


In [141]:
with open("TEXT_DATA", "wb") as fp:   #Pickling
    pickle.dump(TEXT_DATA, fp)

In [91]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_cnn_data[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    8,    3,  399,   32, 1797,
        184,   70,  418, 1557, 5079,   50])

In [62]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(14881, 300)


In [63]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Define RNN-LSTM

In [64]:
data

,Text,Sentiment,Positive,Negative,Neutral,Text_Final,tokens
0,- any day now. When it gets below 61.60.,0.0,0,1,0,day gets 61.60,"[day, gets, 61.60]"
1,"Elephant hunting ISG, PCN, AMZN, A point and...",0.0,0,1,0,elephant hunting isg pcn amzn point figure rev...,"[elephant, hunting, isg, pcn, amzn, point, fig..."
2,FDX double toppoing BEAISH MONTHY GATEY pat...,0.0,0,1,0,fdx double toppoing beaish monthy gatey patter...,"[fdx, double, toppoing, beaish, monthy, gatey,..."
3,ISG An update to our Feb 20th video review.....,0.0,0,1,0,isg update feb 20th video review closes 495 mu...,"[isg, update, feb, 20th, video, review, closes..."
4,----------------------------------------------...,0.5,0,0,1,-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -...,"[--, --, --, --, --, --, --, --, --, --, --, -..."
...,...,...,...,...,...,...,...
10497,"ZOOM long .76, looking for continuation/momo",1.0,1,0,0,zoom long .76 looking continuation/momo,"[zoom, long, .76, looking, continuation/momo]"
10498,"ZOT good entry here, next fib level is 10.39, ...",1.0,1,0,0,zot good entry next fib level 10.39 current su...,"[zot, good, entry, next, fib, level, 10.39, cu..."
10499,ZOT Sold at 10.99 the shares I bought Mar 1 fo...,1.0,1,0,0,zot sold 10.99 shares bought mar 1 8.8,"[zot, sold, 10.99, shares, bought, mar, 1, 8.8]"
10500,ZQK first sign of decreasing volume and 3 (15)...,0.0,0,1,0,zqk first sign decreasing volume 3 15 minute r...,"[zqk, first, sign, decreasing, volume, 3, 15, ..."


In [65]:
label_names =['Positive','Negative','Neutral']

In [67]:
y_train = data_train[label_names].values
x_train = train_cnn_data
y_tr = y_train

In [68]:
def recurrent_nn(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

#     lstm = LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embedded_sequences)
    lstm = LSTM(256)(embedded_sequences)
    
    x = Dense(128, activation='relu')(lstm)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [69]:
model = recurrent_nn(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding (Embedding)       (None, 50, 300)           4464300   
                                                                 
 lstm (LSTM)                 (None, 256)               570368    
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 5,067,951
Trainable params: 603,651
Non-trainab

# Train RNN-LSTM

In [71]:
num_epochs = 9
batch_size = 40
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/9
201/201 [==============================] - 108s 518ms/step - loss: 0.5034 - acc: 0.6116 - val_loss: 0.4739 - val_acc: 0.6372
Epoch 2/9
201/201 [==============================] - 105s 522ms/step - loss: 0.4158 - acc: 0.7010 - val_loss: 0.4095 - val_acc: 0.6920
Epoch 3/9
201/201 [==============================] - 102s 510ms/step - loss: 0.3654 - acc: 0.7437 - val_loss: 0.3909 - val_acc: 0.7279
Epoch 4/9
201/201 [==============================] - 96s 480ms/step - loss: 0.3290 - acc: 0.7788 - val_loss: 0.3874 - val_acc: 0.7189
Epoch 5/9
201/201 [==============================] - 102s 509ms/step - loss: 0.2949 - acc: 0.8016 - val_loss: 0.3841 - val_acc: 0.7458
Epoch 6/9
201/201 [==============================] - 113s 564ms/step - loss: 0.2563 - acc: 0.8336 - val_loss: 0.4125 - val_acc: 0.7324
Epoch 7/9
201/201 [==============================] - 112s 555ms/step - loss: 0.2189 - acc: 0.8603 - val_loss: 0.4092 - val_acc: 0.7312
Epoch 8/9
201/201 [==============================] - 112

### Test RNN_LSTM

In [72]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

2/2 [==============================] - 3s 976ms/step


In [102]:
labels = [1, 0,.5]
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])
# sum(data_test.Sentiment==prediction_labels)/len(prediction_labels)   
prediction_labels

[1,
 0.5,
 1,
 1,
 0.5,
 0.5,
 1,
 0,
 1,
 0,
 1,
 0.5,
 1,
 1,
 0,
 0.5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0.5,
 0.5,
 1,
 1,
 0,
 0.5,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0.5,
 0.5,
 1,
 0.5,
 1,
 0.5,
 0.5,
 0.5,
 1,
 0,
 1,
 1,
 0.5,
 1,
 1,
 1,
 1,
 0.5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0.5,
 0.5,
 0.5,
 0,
 0.5,
 1,
 1,
 1,
 1,
 0.5,
 0,
 0,
 0.5,
 1,
 1,
 0,
 0.5,
 0.5,
 1,
 1,
 1,
 0,
 1,
 0.5,
 1,
 0,
 1,
 0.5,
 0.5,
 1,
 0.5,
 0,
 1,
 1,
 0.5,
 1,
 1,
 1,
 0.5,
 1,
 0.5,
 0.5,
 1,
 0.5,
 0,
 0,
 0,
 0.5,
 1,
 1,
 0.5,
 0.5,
 0,
 0,
 1,
 0,
 1,
 1,
 0.5,
 0.5,
 0,
 0,
 0,
 1,
 0,
 1,
 0.5,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0.5,
 1,
 0,
 1,
 1,
 0.5,
 1,
 0,
 0.5,
 1,
 1,
 1,
 1,
 0.5,
 1,
 0.5,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0.5,
 1,
 0.5,
 0,
 1,
 1,
 1,
 0.5,
 1,
 0,
 0.5,
 1,
 0,
 0,
 1,
 1,
 0.5,
 0,
 0.5,
 1,
 0,
 1,
 0.5,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0.5,
 0,
 1,
 0.5,
 1,
 1,
 1,
 1,
 0.5,
 0.5

In [131]:
inputs=input("Enter the news:")

Enter the news:JSW Steel combined crude steel output grows 37%; FY22 production soars 38%


In [132]:
input_un_tokens = [word_tokenize(inputs)]
remove_sym=[';',':','!','@','#','$','%',',','.','^','&','*','(',')','/','?','`','...','..','--','---','~',"+","=",'-',"}","{"]
for i in input_un_tokens:
    if i in remove_sym:
        input_un_tokens.remove(i)
# input_un_tokens
lower_input_tokens = [lower_token(in_token) for in_token in input_un_tokens] 
# lower_input_tokens
filtered_words = [remove_stop_words(sen) for sen in lower_input_tokens] 
sentance = [' '.join(sen) for sen in filtered_words]
# sentance
input_sequences = tokenizer.texts_to_sequences(sentance)
cnn_data = pad_sequences(input_sequences, maxlen=MAX_SEQUENCE_LENGTH)
# input_sequences
cnn_data

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,  353,  811, 1295,  353,
        1284, 6440,  411,  118, 3675,  354]])

In [133]:
p=model.predict(cnn_data)

In [134]:
print(labels[np.argmax(p)])

1


# Save Model

In [135]:
import os.path
if os.path.isfile('models/sentiment_analysis_stock.h5')is False:
    model.save('models/sentiment_analysis_stock.h5')